In [1]:
# importing libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sqlite3

# importing datasets.

retail = pd.read_csv("data/BrandAverageRetailPrice.csv")
details = pd.read_csv("data/BrandDetails.csv")
sales = pd.read_csv("data/BrandTotalSales.csv")
units = pd.read_csv("data/BrandTotalUnits.csv")

#converting all of our months to datetime
retail['Months'] = pd.to_datetime(retail['Months'])
sales['Months'] = pd.to_datetime(sales['Months'])
units['Months'] = pd.to_datetime(units['Months'])

# Retail dataset

col names = Brands, Months, ARP, "vs. Prior Period"

In [2]:
retail.info()
# ARP has 1939 null entries.
# "vs. Prior Period" has 2712 null entries.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Brands            27211 non-null  object        
 1   Months            27211 non-null  datetime64[ns]
 2   ARP               25279 non-null  float64       
 3   vs. Prior Period  24499 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 850.5+ KB


In [3]:
retail.head()

,Brands,Months,ARP,vs. Prior Period
0,#BlackSeries,2020-08-01,15.684913,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,13.611428,NaN
3,#BlackSeries,2021-02-01,11.873182,-0.127705
4,#BlackSeries,2021-03-01,NaN,-1.000000


# sales dataset
- Need to convert total sales into an int or float.

In [4]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 592.6+ KB


In [5]:
# Concert total sales feature into floats
sales['Total Sales ($)'].replace(',','', regex=True, inplace=True)
sales['Total Sales ($)'] = sales['Total Sales ($)'].str[:8]
sales['Total Sales ($)'] = pd.to_numeric(sales['Total Sales ($)'])

In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25279 entries, 0 to 25278
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Months           25279 non-null  datetime64[ns]
 1   Brand            25279 non-null  object        
 2   Total Sales ($)  25279 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 592.6+ KB


# units dataset
need to convert total units into an int or float

In [7]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Brands            27686 non-null  object        
 1   Months            27686 non-null  datetime64[ns]
 2   Total Units       25712 non-null  object        
 3   vs. Prior Period  24935 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 865.3+ KB


In [8]:
units = units.rename(columns={"Brands": "Brand"})
units = units.rename(columns={"vs. Prior Period": "vs. Prior Period (Units)"})

units['Total Units'].replace(',','', regex=True, inplace=True)
units['Total Units'] = units['Total Units'].str[:8].replace(',','')
units['Total Units'] = pd.to_numeric(units['Total Units'])

In [9]:
units.head(10)

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000
5,101 Cannabis Co.,2019-11-01,131.0677,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000
7,101 Cannabis Co.,2020-01-01,345.4134,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169


In [10]:
units.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27686 entries, 0 to 27685
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     27686 non-null  object        
 1   Months                    27686 non-null  datetime64[ns]
 2   Total Units               25712 non-null  float64       
 3   vs. Prior Period (Units)  24935 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 865.3+ KB


In [11]:
# Merging the datasets (Except for details)

In [12]:
retail = retail.rename(columns={"Brands": "Brand"})
retail = retail.rename(columns={"vs. Prior Period": "vs. Prior Period (ARP)"})
retail.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27211 entries, 0 to 27210
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Brand                   27211 non-null  object        
 1   Months                  27211 non-null  datetime64[ns]
 2   ARP                     25279 non-null  float64       
 3   vs. Prior Period (ARP)  24499 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 850.5+ KB


# Merging all datasets except for Details

- for ARP, 1932 nulls
- for ARP vs prior, 2712 nulls
- for Units, 1932 nulls
- for Units vs prior, 2813 nulls

In [13]:
retailXunits = pd.merge(retail, units, on=['Brand', 'Months'], how='left')
full = pd.merge(retailXunits, sales, on=['Brand', 'Months'], how='left')
full.head()

,Brand,Months,ARP,vs. Prior Period (ARP),Total Units,vs. Prior Period (Units),Total Sales ($)
0,#BlackSeries,2020-08-01,15.684913,NaN,1616.3390,NaN,25352.130
1,#BlackSeries,2020-09-01,NaN,-1.000000,NaN,-1.000000,NaN
2,#BlackSeries,2021-01-01,13.611428,NaN,715.5328,NaN,9739.423
3,#BlackSeries,2021-02-01,11.873182,-0.127705,766.6691,0.071466,9102.802
4,#BlackSeries,2021-03-01,NaN,-1.000000,NaN,-1.000000,NaN


In [14]:
full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27211 entries, 0 to 27210
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     27211 non-null  object        
 1   Months                    27211 non-null  datetime64[ns]
 2   ARP                       25279 non-null  float64       
 3   vs. Prior Period (ARP)    24499 non-null  float64       
 4   Total Units               25279 non-null  float64       
 5   vs. Prior Period (Units)  24893 non-null  float64       
 6   Total Sales ($)           25279 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.7+ MB


## Checking to see what happens if we just drop all rows with NaN values
- We lose 4641 rows, which is ~18% of our dataset

In [15]:
tester = full
tester = tester.dropna()
tester.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22570 entries, 3 to 27209
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Brand                     22570 non-null  object        
 1   Months                    22570 non-null  datetime64[ns]
 2   ARP                       22570 non-null  float64       
 3   vs. Prior Period (ARP)    22570 non-null  float64       
 4   Total Units               22570 non-null  float64       
 5   vs. Prior Period (Units)  22570 non-null  float64       
 6   Total Sales ($)           22570 non-null  float64       
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 1.4+ MB


In [16]:
units.sort_values(by=['Brand','Months'], inplace=True)

In [17]:
units.head(50)

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000
5,101 Cannabis Co.,2019-11-01,131.0677,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000
7,101 Cannabis Co.,2020-01-01,345.4134,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169


In [18]:
sales.describe()

,Total Sales ($)
count,2.527900e+04
mean,4.093728e+05
std,1.596024e+06
min,0.000000e+00
25%,1.390319e+04
50%,6.210080e+04
75%,2.473270e+05
max,4.036351e+07


# Timeseries feature engineering

In [19]:
# starting with a single brand

blah = units[units.Brand == '#BlackSeries'] # where brand == some brand
blah

,Brand,Months,Total Units,vs. Prior Period (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000
2,#BlackSeries,2021-01-01,715.5328,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466
4,#BlackSeries,2021-03-01,NaN,-1.000000


In [20]:
blah = units
blah.loc[:,'Previous Month (Units)'] = blah.loc[:,'Total Units'].shift(+1)
blah.loc[:,'Rolling Average (Units)'] = (blah.loc[:,'Total Units'].shift(+1) + blah.loc[:,'Total Units'].shift(+2) + blah.loc[:,'Total Units'].shift(+3))/3
blah.shape

(27686, 6)

In [21]:
blah.head(50)

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN
5,101 Cannabis Co.,2019-11-01,131.0677,NaN,NaN,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000,131.0677,NaN
7,101 Cannabis Co.,2020-01-01,345.4134,NaN,NaN,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883,345.4134,NaN
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169,696.6584,NaN


In [ ]:
BS = units[units.Brand == '#BlackSeries']


In [22]:
blah.shape

(27686, 6)

In [23]:
l = blah.shape[0]
i = 1
while i < l:
    if blah.iloc[i]['Brand'] != blah.iloc[i-1]['Brand']:
        blah.drop(i, inplace=True)
        l-=1
    i+=1

In [24]:
blah.shape

(26063, 6)

In [25]:
blah.head(50)

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units)
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN
6,101 Cannabis Co.,2019-12-01,NaN,-1.000000,131.0677,NaN
7,101 Cannabis Co.,2020-01-01,345.4134,NaN,NaN,NaN
8,101 Cannabis Co.,2020-02-01,696.6584,1.016883,345.4134,NaN
9,101 Cannabis Co.,2020-03-01,943.3933,0.354169,696.6584,NaN
10,101 Cannabis Co.,2020-04-01,712.4981,-0.244750,943.3933,661.821700


In [26]:
# creating new dataframe from consumption column
#data_historic = units[['Total Units']]
# inserting new column with yesterday's consumption values
blah.loc[:,'Previous Month ()'] = blah.loc[:,'Total Units'].shift(+1)
# inserting another column with difference between yesterday and day before yesterday's consumption values.
blah.loc[:,'Rolling Average'] = (blah.loc[:,'Total Units'].shift(+1) + blah.loc[:,'Total Units'].shift(+2) + blah.loc[:,'Total Units'].shift(+3))/3

blah

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units),Previous Month (),Rolling Average
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN,766.6691,NaN
...,...,...,...,...,...,...,...,...
12660,iKrusher,2020-06-01,671.5098,-0.342416,1021.1760,NaN,1021.1760,1030.507357
12661,iKrusher,2020-07-01,NaN,-1.000000,671.5098,NaN,671.5098,579.406957
12662,iKrusher,2021-07-01,404.2586,NaN,NaN,NaN,NaN,NaN
12663,iKrusher,2021-08-01,987.4264,1.442561,404.2586,NaN,404.2586,NaN


In [27]:
type(blah)

pandas.core.frame.DataFrame

In [28]:
# Generalising the code for the entire units dataset.

brands = units["Brand"].unique()
brands

array(['#BlackSeries', '101 Cannabis Co.', '10x Infused', ..., 'Zoma',
       'Zuma Topicals', 'iKrusher'], dtype=object)

In [29]:
blah

,Brand,Months,Total Units,vs. Prior Period (Units),Previous Month (Units),Rolling Average (Units),Previous Month (),Rolling Average
0,#BlackSeries,2020-08-01,1616.3390,NaN,NaN,NaN,NaN,NaN
1,#BlackSeries,2020-09-01,NaN,-1.000000,1616.3390,NaN,1616.3390,NaN
2,#BlackSeries,2021-01-01,715.5328,NaN,NaN,NaN,NaN,NaN
3,#BlackSeries,2021-02-01,766.6691,0.071466,715.5328,NaN,715.5328,NaN
4,#BlackSeries,2021-03-01,NaN,-1.000000,766.6691,NaN,766.6691,NaN
...,...,...,...,...,...,...,...,...
12660,iKrusher,2020-06-01,671.5098,-0.342416,1021.1760,NaN,1021.1760,1030.507357
12661,iKrusher,2020-07-01,NaN,-1.000000,671.5098,NaN,671.5098,579.406957
12662,iKrusher,2021-07-01,404.2586,NaN,NaN,NaN,NaN,NaN
12663,iKrusher,2021-08-01,987.4264,1.442561,404.2586,NaN,404.2586,NaN


In [30]:
brands.shape

(1639,)

In [31]:
np.unique(brands).shape

(1639,)

In [32]:
#blah = pd.DataFrame()

#for brand in brands:
    #print(brand)
    #brand_df = units[units.Brand == brand]
    #brand_df.loc[:,'Previous Month (Units)'] = brand_df.loc[:,'Total Units'].shift(+1)
    #brand_df.loc[:,'Rolling Average (Units)'] = (brand_df.loc[:,'Total Units'].shift(+1) + brand_df.loc[:,'Total Units'].shift(+2) + brand_df.loc[:,'Total Units'].shift(+3))/3
    #blah.append(brand_df)

#blah

In [33]:
#  

In [34]:
brand_df['Category L1'].value_counts()

NameError: name 'brand_df' is not defined